In [0]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor as gb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn import preprocessing 
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from sklearn import preprocessing
from sklearn.manifold import TSNE

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd 
from imblearn.over_sampling import RandomOverSampler
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bert_serving.client import BertClient

train = pd.read_csv("drive/My Drive/Colab Notebooks/reddit_cnn/data/training_data.csv")
train_emb = pd.read_csv("drive/My Drive/Colab Notebooks/reddit_cnn/data/train_emb.csv")
test_emb = pd.read_csv("drive/My Drive/Colab Notebooks/reddit_cnn/data/test_emb.csv")
# Encoding labels from classification dummies
ls_labels = list()
for i in range(len(train)):
  if train["subredditcars"][i] == 1:
    ls_labels.append(0)
  if train["subredditCooking"][i] == 1:
    ls_labels.append(1)
  if train["subredditMachineLearning"][i] == 1:
    ls_labels.append(2)
  if train["subredditmagicTCG"][i] == 1:
    ls_labels.append(3)
  if train["subredditpolitics"][i] == 1:
    ls_labels.append(4)
  if train["subredditReal_Estate"][i] == 1:
    ls_labels.append(5)
  if train["subredditscience"][i] == 1:
    ls_labels.append(6)
  if train["subredditStockMarket"][i] == 1:
    ls_labels.append(7)
  if train["subreddittravel"][i] == 1:
    ls_labels.append(8)
  if train["subredditvideogames"][i] == 1:
    ls_labels.append(9)
train = train[["id", "text"]]
train["labels"] = pd.to_numeric(ls_labels)
# ros = RandomOverSampler(random_state = 42)

X = train.drop("id", axis = 1)
y = train["labels"]
X = X.drop("labels", axis = 1)
# encoded = client.encode(X["text"])
full_emb = pd.concat([train_emb, test_emb], axis = 0)

# PCA Analysis: n_components =105, because it explains 95% of the variance in our data. 
pca = PCA(105, svd_solver = "randomized").fit(full_emb)

# tsne = TSNE(n_components = 2, perplexity = 50, n_iter=2500)

train_emb = pca.transform(train_emb)
# train_emb = tsne.fit(full_emb).transform(train_emb)
train_emb = pd.DataFrame(train_emb)

ros = RandomOverSampler()
X = X.drop("text", axis = 1)
X = pd.concat([X, train_emb], axis = 1)
X, y = ros.fit_resample(X, y)

X = pd.DataFrame(X)
y = pd.Series(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

KeyboardInterrupt: ignored

In [0]:
from sklearn.utils import class_weight

class_weights_train = list(class_weight.compute_class_weight('balanced',
                                             np.unique(y),
                                             y_train))

w_array_train = np.ones(y_train.shape[0], dtype = 'float')
for i, val in enumerate(y_train):
    w_array_train[i] = class_weights_train[val-1]
w_array_train
class_weights_test = list(class_weight.compute_class_weight('balanced',
                                             np.unique(y),
                                             y_test))

w_array_test = np.ones(y_test.shape[0], dtype = 'float')
for i, val in enumerate(y_test):
    w_array_test[i] = class_weights_test[val-1]

w_array_test



array([0.96      , 0.8       , 0.75789474, 1.10769231, 0.9       ,
       1.10769231, 0.96      , 0.9       , 0.9       , 0.96      ,
       1.30909091, 1.10769231, 1.10769231, 0.9       , 0.9       ,
       0.9       , 0.96      , 0.9       , 1.44      , 0.8       ,
       1.10769231, 0.75789474, 1.30909091, 1.10769231, 0.8       ,
       1.10769231, 0.75789474, 0.8       , 1.10769231, 0.8       ,
       0.9       , 0.9       , 0.75789474, 0.8       , 0.9       ,
       0.8       , 1.10769231, 1.10769231, 1.10769231, 0.96      ,
       0.75789474, 0.75789474, 0.9       , 0.9       , 0.9       ,
       0.9       , 1.10769231, 1.44      , 1.10769231, 1.10769231,
       1.10769231, 1.44      , 1.44      , 1.10769231, 0.96      ,
       0.8       , 1.10769231, 1.10769231, 1.44      , 1.10769231,
       0.96      , 0.75789474, 0.9       , 0.9       , 0.8       ,
       1.10769231, 0.75789474, 0.75789474, 1.30909091, 1.10769231,
       1.30909091, 0.9       , 0.8       , 0.8       , 1.44   

In [0]:
D_train = xgb.DMatrix(X_train, label=y_train, weight = w_array_train)
D_test = xgb.DMatrix(X_test, label=y_test, weight = w_array_trainy_test)
params = {'subsample' : 1.0, 
          'tree_method' : 'hist',
          'eval_metric' : 'mlogloss',
          'reg_lambda': 0, 
          'reg_alpha' : 0, 
          'objective' : 'multi:softprob',
          'num_class' : 10,
          'min_child_weight' : 1, 
          'max_depth' : 20, 
          'grow_policy': 'lossguide',
          'learning_rate' : 0.01, 
          'gamma' : 0.3, 
          'colsample_bytree' : 0.6}

steps = 15000  # The number of training iterations

model = xgb.train(params, dtrain=D_train, num_boost_round=steps, evals=[(D_test, 'eval')],verbose_eval=True, early_stopping_rounds=50)

[0]	eval-mlogloss:2.26064
Will train until eval-mlogloss hasn't improved in 50 rounds.
[1]	eval-mlogloss:2.21767
[2]	eval-mlogloss:2.1781
[3]	eval-mlogloss:2.14099
[4]	eval-mlogloss:2.10387
[5]	eval-mlogloss:2.06911
[6]	eval-mlogloss:2.03506
[7]	eval-mlogloss:2.00337
[8]	eval-mlogloss:1.97146
[9]	eval-mlogloss:1.94142
[10]	eval-mlogloss:1.91255
[11]	eval-mlogloss:1.8841
[12]	eval-mlogloss:1.85672
[13]	eval-mlogloss:1.82984
[14]	eval-mlogloss:1.8035
[15]	eval-mlogloss:1.77821
[16]	eval-mlogloss:1.75375
[17]	eval-mlogloss:1.73042
[18]	eval-mlogloss:1.7072
[19]	eval-mlogloss:1.68427
[20]	eval-mlogloss:1.66213
[21]	eval-mlogloss:1.64063
[22]	eval-mlogloss:1.61899
[23]	eval-mlogloss:1.59894
[24]	eval-mlogloss:1.57865
[25]	eval-mlogloss:1.56062
[26]	eval-mlogloss:1.54194
[27]	eval-mlogloss:1.5227
[28]	eval-mlogloss:1.50395
[29]	eval-mlogloss:1.48563
[30]	eval-mlogloss:1.46759
[31]	eval-mlogloss:1.45018
[32]	eval-mlogloss:1.43304
[33]	eval-mlogloss:1.41645
[34]	eval-mlogloss:1.40014
[35]	eval

In [0]:

test_emb = pca.transform(test_emb)
test_emb = pd.DataFrame(test_emb)
test_emb = xgb.DMatrix(test_emb)


In [0]:
preds = model.predict(test_emb)
examp_sub = pd.read_csv("drive/My Drive/Colab Notebooks/reddit_cnn/data/example_sub.csv")
exam_id = examp_sub.drop("id", axis = 1)
output = pd.DataFrame(preds, columns = exam_id.columns.values.tolist())
output.insert(0, "id", examp_sub["id"])
output

,id,subredditcars,subredditCooking,subredditMachineLearning,subredditmagicTCG,subredditpolitics,subredditReal_Estate,subredditscience,subredditStockMarket,subreddittravel,subredditvideogames
0,test_1,0.028798,0.016917,0.096081,0.028898,0.014364,0.011567,0.741204,0.020404,0.011680,0.030087
1,test_2,0.036110,0.005985,0.134406,0.029128,0.006787,0.004368,0.761841,0.011636,0.004409,0.005330
2,test_3,0.113775,0.040103,0.058712,0.070451,0.013397,0.010801,0.584588,0.014031,0.018231,0.075912
3,test_4,0.979716,0.001326,0.011441,0.001224,0.000766,0.000639,0.001890,0.000761,0.000646,0.001592
4,test_5,0.023567,0.058422,0.036009,0.023476,0.007442,0.005894,0.822396,0.007599,0.007673,0.007522
...,...,...,...,...,...,...,...,...,...,...,...
20550,test_20551,0.237159,0.016277,0.425922,0.038350,0.017284,0.013709,0.144565,0.016882,0.072958,0.016894
20551,test_20552,0.983941,0.004126,0.000876,0.001023,0.000664,0.000530,0.005306,0.000652,0.002246,0.000637
20552,test_20553,0.470306,0.196504,0.018391,0.146411,0.009125,0.007031,0.117298,0.008652,0.007745,0.018537
20553,test_20554,0.980142,0.004385,0.001112,0.003668,0.000908,0.003664,0.003571,0.000885,0.000771,0.000894


In [0]:
output.to_csv("xgboost.csv", index = False)
!cp xgboost.csv "drive/My Drive/Colab Notebooks/reddit_cnn/"